In [1]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import numpy as np
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.tree import DecisionTreeClassifier
import joblib
from sklearn import svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
data=pd.read_csv('data.csv')

In [3]:
data=data.dropna()

In [4]:
data_up=data.groupby('target').apply(lambda x: x.sample(18000))

In [5]:
data_up['target']=data_up['target'].replace({'NoHate':0,'Hate':1})
y=data_up[['target']]


In [6]:
X_train,X_test,Y_train,Y_test=train_test_split(data_up['clean_comment_string1'],y,test_size=0.25,random_state=12,shuffle=True)

In [7]:
vectorizer = CountVectorizer()
#vectorizer.fit()
vector = vectorizer.fit_transform(list(data_up['clean_comment_string1'])[:1000]).toarray()

In [8]:
joblib.dump(vectorizer,'../model/count.pkl')

['../model/count.pkl']

In [9]:
X_train,X_test,Y_train,Y_test=train_test_split(data_up['clean_comment_string1'],y,test_size=0.25,random_state=12,shuffle=True)

In [10]:
pipe_lr = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression())])

In [11]:
type(X_train)

pandas.core.series.Series

In [12]:
pipe_lr.fit(X_train,Y_train)

/Users/kohishaaruganti/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kohishaaruganti/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('cv', CountVectorizer()), ('lr', LogisticRegression())])

In [13]:
pipe_lr.score(X_test,Y_test)

0.8706666666666667

In [14]:
ex1 = "This book was so interesting it made me happy"
pipe_lr.predict([ex1])

array([0])

In [15]:
pipe_lr.predict_proba([ex1])

array([[0.85372907, 0.14627093]])

In [16]:
pipeline_file = open("count_lr.pkl","wb")
joblib.dump(pipe_lr,pipeline_file)
pipeline_file.close()

In [17]:
pipe_dt = Pipeline(steps=[('cv',CountVectorizer()),('lr',DecisionTreeClassifier())])
pipe_dt.fit(X_train,Y_train)
pipe_dt.score(X_test,Y_test)

0.8062222222222222

In [18]:
joblib.dump(pipe_dt,'../model/count_dt.pkl')

['../model/count_dt.pkl']

In [19]:
pipe_svm = Pipeline(steps=[('cv',CountVectorizer()),('lr',svm.SVC(degree=3, gamma='auto',probability=True))])
pipe_svm.fit(X_train,Y_train)
pipe_svm.score(X_test,Y_test)

/Users/kohishaaruganti/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.5237777777777778

In [20]:
joblib.dump(pipe_svm,'../model/count_svm.pkl')

['../model/count_svm.pkl']

In [21]:
pipe_rf = Pipeline(steps=[('cv',CountVectorizer()),('lr',RandomForestClassifier())])
pipe_rf.fit(X_train,Y_train)
pipe_rf.score(X_test,Y_test)
joblib.dump(pipe_rf,'../model/count_rf.pkl')

/Users/kohishaaruganti/opt/anaconda3/lib/python3.9/site-packages/sklearn/pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


['../model/count_rf.pkl']

In [22]:
pipe_gb = Pipeline(steps=[('cv',CountVectorizer()),('lr',GradientBoostingClassifier())])
pipe_gb.fit(X_train,Y_train)
pipe_gb.score(X_test,Y_test)
joblib.dump(pipe_gb,'../model/count_gb.pkl')

/Users/kohishaaruganti/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['../model/count_gb.pkl']

In [23]:
pipe_nb = Pipeline(steps=[('cv',CountVectorizer()),('lr',MultinomialNB())])
pipe_nb.fit(X_train,Y_train)
pipe_nb.score(X_test,Y_test)
joblib.dump(pipe_nb,'../model/count_nb.pkl')

/Users/kohishaaruganti/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['../model/count_nb.pkl']

Word2Vec

In [24]:
sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences,  window=5, min_count=5, workers=4)

In [25]:
joblib.dump(w2v_model,'../model/word2vec.pkl')

['../model/word2vec.pkl']

In [26]:
w2v_model1=joblib.load('../model/word2vec.pkl')

In [27]:
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_trainword2vec = np.array([vectorize(sentence) for sentence in X_train])
X_testword2vec = np.array([vectorize(sentence) for sentence in X_test])

In [28]:
samplworvec=np.array(vectorize('I hate icecream'))
type(samplworvec)

numpy.ndarray

In [29]:
clf = LogisticRegression()
clf.fit(X_trainword2vec, Y_train)

/Users/kohishaaruganti/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/kohishaaruganti/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
y_pred = clf.predict(X_testword2vec)
print('Accuracy:', accuracy_score(Y_test, y_pred))
print('Precision:', precision_score(Y_test, y_pred))
print('Recall:', recall_score(Y_test, y_pred))
print('F1 score:', f1_score(Y_test, y_pred))

Accuracy: 0.8334444444444444
Precision: 0.8428017339721652
Recall: 0.8201598579040853
F1 score: 0.8313266569145944


In [31]:
joblib.dump(clf,'../model/word2veclr.pkl')

['../model/word2veclr.pkl']

In [32]:
clf.predict(samplworvec.reshape(1,-1))
clf.predict_proba(samplworvec.reshape(1,-1))

array([[1.28706629e-06, 9.99998713e-01]])

Word2VecDT

In [33]:
word2vec_dt = DecisionTreeClassifier()
word2vec_dt.fit(X_trainword2vec, Y_train)
y_pred_w2vecdt = word2vec_dt.predict(X_testword2vec)
print('Accuracy:', accuracy_score(Y_test, y_pred_w2vecdt))
print('Precision:', precision_score(Y_test, y_pred_w2vecdt))
print('Recall:', recall_score(Y_test, y_pred_w2vecdt))
print('F1 score:', f1_score(Y_test, y_pred_w2vecdt))

Accuracy: 0.7574444444444445
Precision: 0.7539943094769096
Recall: 0.7648756660746003
F1 score: 0.7593960101399758


In [34]:
samplworvec=np.array(vectorize('I hate icecream'))
type(samplworvec)

numpy.ndarray

In [35]:
word2vec_dt.predict_proba([samplworvec])

array([[0., 1.]])

In [36]:
joblib.dump(word2vec_dt,'../model/word2vecdt.pkl')

['../model/word2vecdt.pkl']

In [37]:
word2vec_svm = svm.SVC(degree=3, gamma='auto',probability=True)
word2vec_svm.fit(X_trainword2vec, Y_train)
y_pred_w2vecsvm = word2vec_svm.predict(X_testword2vec)
print('Accuracy:', accuracy_score(Y_test, y_pred_w2vecsvm))
print('Precision:', precision_score(Y_test, y_pred_w2vecsvm))
print('Recall:', recall_score(Y_test, y_pred_w2vecsvm))
print('F1 score:', f1_score(Y_test, y_pred_w2vecsvm))


/Users/kohishaaruganti/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.829
Precision: 0.8246113422377929
Recall: 0.8361456483126111
F1 score: 0.8303384411861977


In [38]:
joblib.dump(word2vec_svm,'../model/word2vec_svm.pkl')

['../model/word2vec_svm.pkl']

In [39]:
word2vec_gb = GradientBoostingClassifier()
word2vec_gb.fit(X_trainword2vec, Y_train)
y_pred_w2vecgb = word2vec_gb.predict(X_testword2vec)
print('Accuracy:', accuracy_score(Y_test, y_pred_w2vecgb))
print('Precision:', precision_score(Y_test, y_pred_w2vecgb))
print('Recall:', recall_score(Y_test, y_pred_w2vecgb))
print('F1 score:', f1_score(Y_test, y_pred_w2vecgb))
joblib.dump(word2vec_gb,'../model/word2vec_gb.pkl')

/Users/kohishaaruganti/opt/anaconda3/lib/python3.9/site-packages/sklearn/ensemble/_gb.py:437: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy: 0.8324444444444444
Precision: 0.8207708779443255
Recall: 0.8510213143872114
F1 score: 0.8356224111619796


['../model/word2vec_gb.pkl']